<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/zazuko/notebooks/blob/master/notebooks/politics/parliament.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img/colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/zazuko/notebooks/master/notebooks/politics/parliament.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/zazuko/notebooks/blob/master/notebooks/politics/parliament.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Swiss Parliament: Comissions

Federal Chancellery maintains data on swiss political system. The curia dataset is publicly available. It provides us with political parties, parliamentary comissions, members of parliament and their affiliations. 

Parliament data is also available as [Linked Data](https://en.wikipedia.org/wiki/Linked_data). 

## Setup

### SPARQL endpoint

Swiss political data can be accessed with [SPARQL queries](https://www.w3.org/TR/rdf-sparql-query/).   
You can send queries using HTTP requests. The API endpoint is **[https://lindas.admin.ch/query/](https://lindas.admin.ch/query).**  

### SPARQL client

Let's use `SparqlClient` from [graphly](https://github.com/zazuko/graphly) to communicate with the database. 
Graphly will allow us to:
* send SPARQL queries
* automatically add prefixes to all queries
* format response to `pandas` or `geopandas`

In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

from graphly.api_client import SparqlClient

%matplotlib inline
pd.set_option('display.max_rows', 100)

In [ ]:
# Uncomment to install dependencies in Colab environment
#!pip install git+https://github.com/zazuko/graphly.git

In [9]:
sparql = SparqlClient("https://int.lindas.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>"
})

SPARQL queries can become very long. To improve the readibility, we will work wih [prefixes](https://en.wikibooks.org/wiki/SPARQL/Prefixes).
 
Using the `add_prefixes` method, we can define persistent prefixes. 
Every time you send a query, `graphly` will now automatically add the prefixes for you.

# Political Parties

In [33]:
query = """
SELECT ?council ?faction (COUNT(?mep) AS ?members)
FROM <https://lindas.admin.ch/fch/curia>
FROM <https://lindas.admin.ch/fch/rvov>
WHERE {

  	VALUES ?_council { <https://politics.ld.admin.ch/council/S> <https://politics.ld.admin.ch/council/N> <https://politics.ld.admin.ch/council/UFA>}
	?mep a schema:Person;
         schema:memberOf/schema:memberOf ?_council;
                        schema:memberOf ?role.
  
   FILTER NOT EXISTS { ?role schema:endDate ?end }
  
   ?role schema:memberOf ?_faction.
   ?_faction a <http://schema.org/ParliamentaryGroup>;
             schema:name ?faction.
  
   ?_council schema:name ?council.
   FILTER (lang(?council) = 'de')
   FILTER (lang(?faction) = 'de')
}
GROUP BY ?council ?faction
ORDER BY DESC(?council) DESC (?members)
"""

df = sparql.send_query(query)
df

,council,faction,members
0,Vereinigte Bundesversammlung,Fraktion der Schweizerischen Volkspartei,62
1,Vereinigte Bundesversammlung,Sozialdemokratische Fraktion,47
2,Vereinigte Bundesversammlung,Die Mitte-Fraktion. Die Mitte. EVP.,45
3,Vereinigte Bundesversammlung,FDP-Liberale Fraktion,41
4,Vereinigte Bundesversammlung,Grüne Fraktion,35
5,Vereinigte Bundesversammlung,Grünliberale Fraktion,16
6,Ständerat,Die Mitte-Fraktion. Die Mitte. EVP.,14
7,Ständerat,FDP-Liberale Fraktion,12
8,Ständerat,Sozialdemokratische Fraktion,8
9,Ständerat,Fraktion der Schweizerischen Volkspartei,7


In [34]:
fig = make_subplots(rows=1, cols=3, subplot_titles=df["council"].unique(), specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])

for i, council in enumerate(df["council"].unique()):
     fig.add_trace(go.Pie(
          values=df[df.council == council]["members"],
          labels=df[df.council == council]["faction"]
          ), row=1, col=i+1)
     fig.update_traces(textinfo='none')

fig.update_annotations(yshift=-280)
fig.update_layout(height=400, title={"text": "Members of Parliament", "x": 0.5})
fig.show()

# Parliamentary Committees

In [35]:
query = """
SELECT ?council ?committee (COUNT(?mep) AS ?members)
FROM <https://lindas.admin.ch/fch/curia>
FROM <https://lindas.admin.ch/fch/rvov>
WHERE {

  	VALUES ?_council { <https://politics.ld.admin.ch/council/S> <https://politics.ld.admin.ch/council/N>}
	?mep a schema:Person;
         schema:memberOf/schema:memberOf ?_council;
                        schema:memberOf ?role.
  
   FILTER NOT EXISTS { ?role schema:endDate ?end }
  
   ?role schema:memberOf ?_committee.
   ?_committee a <http://schema.org/ParliamentaryCommittee>;
             schema:additionalType schema:MainCommittee;
             schema:name ?committee.
  
   ?_council schema:name ?council.
   FILTER (lang(?council) = 'de')
   FILTER (lang(?committee) = 'de')
}
GROUP BY ?council ?committee
ORDER BY DESC (?members)
"""

df = sparql.send_query(query)
df

,council,committee,members
0,Nationalrat,"Kommission für Wissenschaft, Bildung und Kultu...",25
1,Nationalrat,Aussenpolitische Kommission NR,25
2,Nationalrat,Staatspolitische Kommission NR,25
3,Nationalrat,Geschäftsprüfungskommission NR,25
4,Nationalrat,Finanzkommission NR,25
5,Nationalrat,Kommission für soziale Sicherheit und Gesundhe...,25
6,Nationalrat,"Kommission für Umwelt, Raumplanung und Energie NR",25
7,Nationalrat,Büro NR,17
8,Ständerat,Finanzkommission SR,13
9,Ständerat,Kommission für soziale Sicherheit und Gesundhe...,13


In [36]:
px.treemap(df, path=["council", "committee"], values="members")

In [37]:
query = """
SELECT ?council ?faction (COUNT(?mep) AS ?members)
FROM <https://lindas.admin.ch/fch/curia>
FROM <https://lindas.admin.ch/fch/rvov>
WHERE {

  	VALUES ?_council { <https://politics.ld.admin.ch/council/S> <https://politics.ld.admin.ch/council/N> <https://politics.ld.admin.ch/council/UFA>}
	?mep a schema:Person;
         schema:memberOf/schema:memberOf ?_council;
                        schema:memberOf ?role.
  
   FILTER NOT EXISTS { ?role schema:endDate ?end }
  
   ?role schema:memberOf ?_faction.
   ?_faction a <http://schema.org/ParliamentaryGroup>;
             schema:name ?faction.
  
   ?_council schema:name ?council.
   FILTER (lang(?council) = 'de')
   FILTER (lang(?faction) = 'de')
}
GROUP BY ?council ?faction
ORDER BY DESC(?council) DESC (?members)
"""

df = sparql.send_query(query)
df

,council,faction,members
0,Vereinigte Bundesversammlung,Fraktion der Schweizerischen Volkspartei,62
1,Vereinigte Bundesversammlung,Sozialdemokratische Fraktion,47
2,Vereinigte Bundesversammlung,Die Mitte-Fraktion. Die Mitte. EVP.,45
3,Vereinigte Bundesversammlung,FDP-Liberale Fraktion,41
4,Vereinigte Bundesversammlung,Grüne Fraktion,35
5,Vereinigte Bundesversammlung,Grünliberale Fraktion,16
6,Ständerat,Die Mitte-Fraktion. Die Mitte. EVP.,14
7,Ständerat,FDP-Liberale Fraktion,12
8,Ständerat,Sozialdemokratische Fraktion,8
9,Ständerat,Fraktion der Schweizerischen Volkspartei,7


# Comissions and Factions

In [38]:
# https://s.zazuko.com/eNjwuw

query = """
SELECT ?committee ?faction (COUNT(?mep) AS ?members)
FROM <https://lindas.admin.ch/fch/curia>
FROM <https://lindas.admin.ch/fch/rvov>

WHERE {

	?mep a schema:Person;
         schema:memberOf/schema:memberOf <https://politics.ld.admin.ch/council/UFA>;
                        schema:memberOf ?role_committee;
                        schema:memberOf ?role_faction.
  
   FILTER NOT EXISTS { ?role_faction schema:endDate ?end }
  
   ?role_faction schema:memberOf ?_faction.
   ?_faction a <http://schema.org/ParliamentaryGroup>;
             schema:name ?faction.
  
   ?role_committee schema:memberOf ?_committee.
   ?_committee a schema:ParliamentaryCommittee;
             schema:additionalType schema:MainCommittee;
             schema:name ?committee.
  
   FILTER (lang(?committee) = 'de')
   FILTER (lang(?faction) = 'de')
}
GROUP BY ?committee ?faction
ORDER BY ?committee ?faction DESC (?members)
"""

df = sparql.send_query(query)
df

,committee,faction,members
0,Aussenpolitische Kommission NR,Die Mitte-Fraktion. Die Mitte. EVP.,4
1,Aussenpolitische Kommission NR,FDP-Liberale Fraktion,4
2,Aussenpolitische Kommission NR,Fraktion der Schweizerischen Volkspartei,6
3,Aussenpolitische Kommission NR,Grüne Fraktion,4
4,Aussenpolitische Kommission NR,Grünliberale Fraktion,2
5,Aussenpolitische Kommission NR,Sozialdemokratische Fraktion,5
6,Büro NR,Die Mitte-Fraktion. Die Mitte. EVP.,4
7,Büro NR,FDP-Liberale Fraktion,1
8,Büro NR,Fraktion der Schweizerischen Volkspartei,3
9,Büro NR,Grüne Fraktion,4


In [39]:
mapping = {"Finanzkommission NR": "Finanzkommissionen",
"Finanzkommission SR": "Finanzkommissionen",
"Geschäftsprüfungskommission NR": "Geschäftsprüfungskommissionen",
"Geschäftsprüfungskommission SR": "Geschäftsprüfungskommissionen",
"Kommission für soziale Sicherheit und Gesundheit NR": "Kommissionen für soziale Sicherheit und Gesundheit",
"Kommission für soziale Sicherheit und Gesundheit SR": "Kommissionen für soziale Sicherheit und Gesundheit"}

df.replace(mapping)

,committee,faction,members
0,Aussenpolitische Kommission NR,Die Mitte-Fraktion. Die Mitte. EVP.,4
1,Aussenpolitische Kommission NR,FDP-Liberale Fraktion,4
2,Aussenpolitische Kommission NR,Fraktion der Schweizerischen Volkspartei,6
3,Aussenpolitische Kommission NR,Grüne Fraktion,4
4,Aussenpolitische Kommission NR,Grünliberale Fraktion,2
5,Aussenpolitische Kommission NR,Sozialdemokratische Fraktion,5
6,Büro NR,Die Mitte-Fraktion. Die Mitte. EVP.,4
7,Büro NR,FDP-Liberale Fraktion,1
8,Büro NR,Fraktion der Schweizerischen Volkspartei,3
9,Büro NR,Grüne Fraktion,4
